In [2]:
!pip install pandas-ta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 40.8 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.4
    Uninstalling tqdm-4.66.4:
      Successfully uninstalled tqdm-4.66.4
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: pandas━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/5 [numpy]
    Found existing installation: pandas 2.2.3━━━━━━━━━━━━━━━━━ 1/5 [numpy]
    Uninstalling pandas-2.2.3:m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/5 [numpy]
      Successfully uninstalled pandas-2.2.3━━━━━━━━━━━━━━━━━━━ 1/5 [numpy]
  Attempting uninstall: numba╺━━━━━━━━━━━━━━━━━━━━━━━ 2/5 [pandas]
    Found existing installation: numba 0.61.0m━━━━━━━━━━━━━━━ 3/5 [numba]
    Uninstalling numb

In [3]:
from pathlib import Path
import pandas as pd
import numpy as np
import re
import os
import pandas_ta as ta

In [9]:
filepath = Path("data/price/VNI_020114_141125.csv")

In [5]:
COLS_TO_EXTRACT_STOCK = ['Lần cuối', 'Mở', 'Cao', 'Thấp', 'KL', '% Thay đổi']
COLS_ENGLISH_NAMES = ['Close', 'Open', 'High', 'Low', 'Volume', 'Change_Pct']

In [6]:
def to_number(x):
    if pd.isna(x): return pd.NA
    s = str(x).strip().replace(',', '')
    try: return float(s)
    except: return pd.NA

def parse_percent(x):
    if pd.isna(x): return pd.NA
    s = str(x).strip().replace(',', '').replace('%', '').replace('−', '-')
    try: return float(s)
    except: return pd.NA

def parse_volume(x):
    if pd.isna(x): return pd.NA
    s = str(x).strip().replace(',', '').upper()
    if 'B' in s: return float(s.replace('B', '')) * 1e9
    if 'M' in s: return float(s.replace('M', '')) * 1e6
    if 'K' in s: return float(s.replace('K', '')) * 1e3
    try: return float(s)
    except: return pd.NA

def clean_and_rename_df(df, original_cols, new_names):
    df_cleaned = df.copy()
    rename_map = dict(zip(original_cols, new_names))
    if 'Lần cuối' in df_cleaned.columns: df_cleaned['Lần cuối'] = df_cleaned['Lần cuối'].apply(to_number)
    if 'Mở' in df_cleaned.columns: df_cleaned['Mở'] = df_cleaned['Mở'].apply(to_number)
    if 'Cao' in df_cleaned.columns: df_cleaned['Cao'] = df_cleaned['Cao'].apply(to_number)
    if 'Thấp' in df_cleaned.columns: df_cleaned['Thấp'] = df_cleaned['Thấp'].apply(to_number)
    if 'KL' in df_cleaned.columns: df_cleaned['KL'] = df_cleaned['KL'].apply(parse_volume)
    if '% Thay đổi' in df_cleaned.columns: df_cleaned['% Thay đổi'] = df_cleaned['% Thay đổi'].apply(parse_percent)
    df_cleaned = df_cleaned.rename(columns=rename_map)
    return df_cleaned
    

In [7]:
def read_and_standardize(path, is_external=False):
    try: df = pd.read_csv(path, encoding='utf-8-sig')
    except Exception: df = pd.read_csv(path, encoding='latin-1')
    if 'Ngày' not in df.columns: return None
    df = df.rename(columns={'Ngày': 'Date'})
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
    df = df.dropna(subset=['Date'])
    if is_external:
        value_col_name = next((col for col in ['Giá', 'Lần cuối', 'Value'] if col in df.columns), None)
        if value_col_name:
            df = df[['Date', value_col_name]].rename(columns={value_col_name: 'Value'})
            df['Value'] = df['Value'].apply(to_number)
        else: return None
    else:
        keep_cols = ['Date'] + [c for c in COLS_TO_EXTRACT_STOCK if c in df.columns]
        df = df[keep_cols]
        df = clean_and_rename_df(df, COLS_TO_EXTRACT_STOCK, COLS_ENGLISH_NAMES)
    return df.sort_values('Date').reset_index(drop=True)

def add_prefix(df, prefix):
    return df.rename(columns={c: f'{prefix}_{c}' for c in df.columns if c != 'Date'})

In [10]:
df = read_and_standardize(filepath)

In [ ]:
# 1. RSI (Relative Strenght Index)

# 2. MACD (Moving Average Convergence Divergence)

# 3. SMA (Simple Moving Average) - 2 đường

# 4. EMA (Exponential Moving Average)

# 5. BBANDS (Bollinger Bands)

# 6. OBV (On-Balance Volume)

# 7. ADX (Average Directional Index)

# 8. STOCH (Stochastic Oscillator)

# 9. ATR (Average True Range)

,Date,Close,Open,High,Low,Volume,Change_Pct
2956,2025-11-10,1580.54,1599.10,1609.49,1580.54,656340000.0,-1.16
2957,2025-11-11,1593.61,1580.54,1597.08,1578.42,596030000.0,0.83
2958,2025-11-12,1631.86,1593.61,1631.86,1593.61,682210000.0,2.40
2959,2025-11-13,1631.44,1631.86,1638.98,1618.77,629950000.0,-0.03
2960,2025-11-14,1634.06,1625.55,1638.35,1623.03,703910000.0,0.16


In [ ]:
# ============================================================================
# 1. RSI (Relative Strength Index)
# ============================================================================
# Purpose: Measures the speed and magnitude of price changes to identify overbought/oversold conditions
# Formula:
#   RSI = 100 - (100 / (1 + RS))
#   where RS = Average Gain / Average Loss over n periods (typically 14 days)
#   - Average Gain = Sum of gains over period / n
#   - Average Loss = Sum of losses over period / n
# Interpretation:
#   - RSI > 70: Overbought (potential sell signal)
#   - RSI < 30: Oversold (potential buy signal)
# ============================================================================

df['RSI_14'] = ta.rsi(df['Close'], length=14)

print("RSI (14-period) calculated")
print(f"Latest RSI: {df['RSI_14'].iloc[-1]:.2f}")
print(f"Overbought (>70): {df['RSI_14'].iloc[-1] > 70}")
print(f"Oversold (<30): {df['RSI_14'].iloc[-1] < 30}")


RSI (14-period) calculated
Latest RSI: 46.65
Overbought (>70): False
Oversold (<30): False


In [14]:
# ============================================================================
# 2. MACD (Moving Average Convergence Divergence)
# ============================================================================
# Purpose: Trend-following momentum indicator showing relationship between two moving averages
# Formula:
#   MACD Line = 12-period EMA - 26-period EMA
#   Signal Line = 9-period EMA of MACD Line
#   MACD Histogram = MACD Line - Signal Line
# Interpretation:
#   - MACD crosses above Signal: Bullish signal (buy)
#   - MACD crosses below Signal: Bearish signal (sell)
#   - Histogram shows momentum strength
# ============================================================================

macd = ta.macd(df['Close'], fast=12, slow=26, signal=9)
df['MACD'] = macd['MACD_12_26_9']
df['MACD_Signal'] = macd['MACDs_12_26_9']
df['MACD_Hist'] = macd['MACDh_12_26_9']

print("MACD calculated")
print(f"Latest MACD: {df['MACD'].iloc[-1]:.2f}")
print(f"Latest Signal: {df['MACD_Signal'].iloc[-1]:.2f}")
print(f"Latest Histogram: {df['MACD_Hist'].iloc[-1]:.2f}")
print(f"Bullish (MACD > Signal): {df['MACD'].iloc[-1] > df['MACD_Signal'].iloc[-1]}")


MACD calculated
Latest MACD: -16.78
Latest Signal: -14.14
Latest Histogram: -2.64
Bullish (MACD > Signal): False


In [24]:
# ============================================================================
# 3. SMA (Simple Moving Average)
# ============================================================================
# Purpose: Smooths price data to identify trend direction
# Formula:
#   SMA = Sum of closing prices over n periods / n
#   - Short-term: 20 days (more responsive to price changes)
#   - Long-term: 50 days (smoother, shows overall trend)
# Interpretation:
#   - Price above SMA: Uptrend
#   - Price below SMA: Downtrend
#   - SMA_20 crosses above SMA_50: Golden Cross (bullish)
#   - SMA_20 crosses below SMA_50: Death Cross (bearish)
# ============================================================================

df['SMA_20'] = ta.sma(df['Close'], length=20)
df['SMA_50'] = ta.sma(df['Close'], length=50)

print("SMA calculated")
print(f"Latest SMA_20: {df['SMA_20'].iloc[-1]:.2f}")
print(f"Latest SMA_50: {df['SMA_50'].iloc[-1]:.2f}")
print(f"Price above SMA_20: {df['Close'].iloc[-1] > df['SMA_20'].iloc[-1]}")
print(f"Golden Cross (SMA_20 > SMA_50): {df['SMA_20'].iloc[-1] > df['SMA_50'].iloc[-1]}")


SMA calculated
Latest SMA_20: 1645.69
Latest SMA_50: 1667.50
Price above SMA_20: False
Golden Cross (SMA_20 > SMA_50): False


In [ ]:
# ============================================================================
# 4. EMA (Exponential Moving Average)
# ============================================================================
# Purpose: Similar to SMA but gives more weight to recent prices
# Formula:
#   EMA = (Close - EMA_prev) × multiplier + EMA_prev
#   where multiplier = 2 / (length + 1)
#   - First EMA = SMA for the period
# Interpretation:
#   - More responsive to recent price changes than SMA
#   - EMA_12 crossing EMA_26 is used in MACD calculation
#   - Price crossing EMA can signal trend changes
# ============================================================================

df['EMA_12'] = ta.ema(df['Close'], length=12)
df['EMA_26'] = ta.ema(df['Close'], length=26)

print("EMA calculated")
print(f"Latest EMA_12: {df['EMA_12'].iloc[-1]:.2f}")
print(f"Latest EMA_26: {df['EMA_26'].iloc[-1]:.2f}")
print(f"Price above EMA_12: {df['Close'].iloc[-1] > df['EMA_12'].iloc[-1]}")
print(f"EMA_12 above EMA_26 (Bullish): {df['EMA_12'].iloc[-1] > df['EMA_26'].iloc[-1]}")


In [ ]:
# ============================================================================
# 5. BBANDS (Bollinger Bands)
# ============================================================================
# Purpose: Measures market volatility and identifies overbought/oversold conditions
# Formula:
#   Middle Band = 20-period SMA
#   Upper Band = Middle Band + (2 × Standard Deviation)
#   Lower Band = Middle Band - (2 × Standard Deviation)
#   Bandwidth = (Upper Band - Lower Band) / Middle Band
# Interpretation:
#   - Price near upper band: Overbought
#   - Price near lower band: Oversold
#   - Narrow bands: Low volatility (potential breakout)
#   - Wide bands: High volatility
#   - Price breaking above/below bands: Strong momentum
# ============================================================================

bbands = ta.bbands(df['Close'], length=20, std=2)
df['BB_Upper'] = bbands['BBU_20_2.0']
df['BB_Middle'] = bbands['BBM_20_2.0']
df['BB_Lower'] = bbands['BBL_20_2.0']
df['BB_Bandwidth'] = bbands['BBB_20_2.0']

print("Bollinger Bands calculated")
print(f"Latest Upper Band: {df['BB_Upper'].iloc[-1]:.2f}")
print(f"Latest Middle Band: {df['BB_Middle'].iloc[-1]:.2f}")
print(f"Latest Lower Band: {df['BB_Lower'].iloc[-1]:.2f}")
print(f"Current Price: {df['Close'].iloc[-1]:.2f}")
print(f"Bandwidth (volatility): {df['BB_Bandwidth'].iloc[-1]:.2f}%")


In [15]:
# ============================================================================
# 6. OBV (On-Balance Volume)
# ============================================================================
# Purpose: Uses volume flow to predict price changes
# Formula:
#   If Close > Close_prev: OBV = OBV_prev + Volume
#   If Close < Close_prev: OBV = OBV_prev - Volume
#   If Close = Close_prev: OBV = OBV_prev
# Interpretation:
#   - Rising OBV: Buying pressure (bullish)
#   - Falling OBV: Selling pressure (bearish)
#   - OBV confirms price trend or shows divergence
#   - Divergence between OBV and price can signal reversal
# ============================================================================

df['OBV'] = ta.obv(df['Close'], df['Volume'])

print("OBV calculated")
print(f"Latest OBV: {df['OBV'].iloc[-1]:,.0f}")
print(f"OBV change (last 5 days): {df['OBV'].iloc[-1] - df['OBV'].iloc[-6]:,.0f}")
print(f"OBV trend (rising): {df['OBV'].iloc[-1] > df['OBV'].iloc[-6]}")


OBV calculated
Latest OBV: 43,859,161,810
OBV change (last 5 days): 695,860,000
OBV trend (rising): True


In [16]:
# ============================================================================
# 7. ADX (Average Directional Index)
# ============================================================================
# Purpose: Measures trend strength (not direction)
# Formula:
#   1. Calculate +DM and -DM (Directional Movement)
#   2. Calculate +DI and -DI (Directional Indicators):
#      +DI = 100 × EMA(+DM) / ATR
#      -DI = 100 × EMA(-DM) / ATR
#   3. Calculate DX = 100 × |+DI - -DI| / (+DI + -DI)
#   4. ADX = EMA of DX over 14 periods
# Interpretation:
#   - ADX < 20: Weak trend (ranging market)
#   - ADX 20-40: Moderate trend
#   - ADX > 40: Strong trend
#   - ADX > 50: Very strong trend
#   - Rising ADX: Strengthening trend
# ============================================================================

adx_data = ta.adx(df['High'], df['Low'], df['Close'], length=14)
df['ADX'] = adx_data['ADX_14']
df['DI_Plus'] = adx_data['DMP_14']
df['DI_Minus'] = adx_data['DMN_14']

print("ADX calculated")
print(f"Latest ADX: {df['ADX'].iloc[-1]:.2f}")
print(f"Latest +DI: {df['DI_Plus'].iloc[-1]:.2f}")
print(f"Latest -DI: {df['DI_Minus'].iloc[-1]:.2f}")
if df['ADX'].iloc[-1] < 20:
    trend_strength = "Weak/Ranging"
elif df['ADX'].iloc[-1] < 40:
    trend_strength = "Moderate"
elif df['ADX'].iloc[-1] < 50:
    trend_strength = "Strong"
else:
    trend_strength = "Very Strong"
print(f"Trend Strength: {trend_strength}")
print(f"Trend Direction: {'Bullish' if df['DI_Plus'].iloc[-1] > df['DI_Minus'].iloc[-1] else 'Bearish'}")


ADX calculated
Latest ADX: 30.21
Latest +DI: 17.04
Latest -DI: 29.32
Trend Strength: Moderate
Trend Direction: Bearish


In [17]:
# ============================================================================
# 8. STOCH (Stochastic Oscillator)
# ============================================================================
# Purpose: Compares closing price to price range over time
# Formula:
#   %K = 100 × (Close - Lowest Low) / (Highest High - Lowest Low)
#   %D = 3-period SMA of %K (signal line)
#   where:
#   - Lowest Low = lowest price over 14 periods
#   - Highest High = highest price over 14 periods
# Interpretation:
#   - %K or %D > 80: Overbought (potential sell)
#   - %K or %D < 20: Oversold (potential buy)
#   - %K crosses above %D: Bullish signal
#   - %K crosses below %D: Bearish signal
# ============================================================================

stoch = ta.stoch(df['High'], df['Low'], df['Close'], k=14, d=3)
df['STOCH_K'] = stoch['STOCHk_14_3_3']
df['STOCH_D'] = stoch['STOCHd_14_3_3']

print("Stochastic Oscillator calculated")
print(f"Latest %K: {df['STOCH_K'].iloc[-1]:.2f}")
print(f"Latest %D: {df['STOCH_D'].iloc[-1]:.2f}")
print(f"Overbought (>80): {df['STOCH_K'].iloc[-1] > 80}")
print(f"Oversold (<20): {df['STOCH_K'].iloc[-1] < 20}")
print(f"Bullish (%K > %D): {df['STOCH_K'].iloc[-1] > df['STOCH_D'].iloc[-1]}")


Stochastic Oscillator calculated
Latest %K: 43.64
Latest %D: 31.62
Overbought (>80): False
Oversold (<20): False
Bullish (%K > %D): True


In [ ]:
# ============================================================================
# 9. ATR (Average True Range)
# ============================================================================
# Purpose: Measures market volatility (not direction)
# Formula:
#   True Range (TR) = max of:
#     1. High - Low
#     2. |High - Close_prev|
#     3. |Low - Close_prev|
#   ATR = EMA of TR over 14 periods
# Interpretation:
#   - High ATR: High volatility (larger price movements)
#   - Low ATR: Low volatility (smaller price movements)
#   - Used for stop-loss placement and position sizing
#   - Not directional - only measures movement magnitude
# ============================================================================

df['ATR_14'] = ta.atr(df['High'], df['Low'], df['Close'], length=14)

print("ATR calculated")
print(f"Latest ATR: {df['ATR_14'].iloc[-1]:.2f}")
print(f"ATR as % of price: {(df['ATR_14'].iloc[-1] / df['Close'].iloc[-1] * 100):.2f}%")
print(f"ATR trend (increasing volatility): {df['ATR_14'].iloc[-1] > df['ATR_14'].iloc[-6]}")


In [ ]:
# Display all calculated indicators
print("\n" + "="*80)
print("SUMMARY OF ALL TECHNICAL INDICATORS")
print("="*80)
print(f"\nDate: {df['Date'].iloc[-1].strftime('%Y-%m-%d')}")
print(f"Close Price: {df['Close'].iloc[-1]:,.2f}")
print("\n--- Momentum Indicators ---")
print(f"RSI (14): {df['RSI_14'].iloc[-1]:.2f}")
print(f"MACD: {df['MACD'].iloc[-1]:.2f} | Signal: {df['MACD_Signal'].iloc[-1]:.2f}")
print(f"Stochastic %K: {df['STOCH_K'].iloc[-1]:.2f} | %D: {df['STOCH_D'].iloc[-1]:.2f}")

print("\n--- Trend Indicators ---")
print(f"SMA (20): {df['SMA_20'].iloc[-1]:,.2f} | SMA (50): {df['SMA_50'].iloc[-1]:,.2f}")
print(f"EMA (12): {df['EMA_12'].iloc[-1]:,.2f} | EMA (26): {df['EMA_26'].iloc[-1]:,.2f}")
print(f"ADX: {df['ADX'].iloc[-1]:.2f} | +DI: {df['DI_Plus'].iloc[-1]:.2f} | -DI: {df['DI_Minus'].iloc[-1]:.2f}")

print("\n--- Volatility Indicators ---")
print(f"Bollinger Upper: {df['BB_Upper'].iloc[-1]:,.2f}")
print(f"Bollinger Middle: {df['BB_Middle'].iloc[-1]:,.2f}")
print(f"Bollinger Lower: {df['BB_Lower'].iloc[-1]:,.2f}")
print(f"ATR (14): {df['ATR_14'].iloc[-1]:.2f}")

print("\n--- Volume Indicators ---")
print(f"OBV: {df['OBV'].iloc[-1]:,.0f}")

print("\n" + "="*80)


In [18]:
# View the dataframe with all indicators
df.tail(10)


,Date,Close,Open,High,Low,Volume,Change_Pct,RSI_14,MACD,MACD_Signal,MACD_Hist,OBV,ADX,DI_Plus,DI_Minus,STOCH_K,STOCH_D
2951,2025-11-03,1617.00,1639.65,1652.48,1614.65,9.652300e+08,-1.38,38.605856,-7.672675,2.828122,-10.500797,4.278694e+10,23.519312,15.194007,31.428541,17.635922,28.280559
2952,2025-11-04,1651.98,1617.00,1658.93,1600.56,1.130000e+09,2.16,45.914280,-8.429470,0.576604,-9.006073,4.391694e+10,24.576657,13.692861,30.708356,16.536651,21.364294
2953,2025-11-05,1654.89,1651.98,1662.96,1640.51,6.148500e+08,0.18,46.485009,-8.694201,-1.277557,-7.416644,4.453179e+10,25.397674,13.860264,29.501098,21.380544,18.517706
2954,2025-11-06,1642.64,1654.89,1659.81,1638.12,5.501900e+08,-0.74,44.362772,-9.779740,-2.977994,-6.801747,4.398160e+10,26.207025,13.315596,28.774802,30.676618,22.864604
2955,2025-11-07,1599.10,1642.64,1644.43,1595.34,8.183000e+08,-2.65,37.763574,-13.992060,-5.180807,-8.811253,4.316330e+10,27.706483,12.151696,33.876961,22.154044,24.737069
2956,2025-11-10,1580.54,1599.10,1609.49,1580.54,6.563400e+08,-1.16,35.349607,-18.613427,-7.867331,-10.746096,4.250696e+10,29.317868,11.512596,34.783970,11.910252,21.580305
2957,2025-11-11,1593.61,1580.54,1597.08,1578.42,5.960300e+08,0.83,38.338790,-20.979417,-10.489748,-10.489668,4.310299e+10,30.845697,11.107106,33.958987,5.262145,13.108814
2958,2025-11-12,1631.86,1593.61,1631.86,1593.61,6.822100e+08,2.40,46.181301,-19.542748,-12.300348,-7.242400,4.378520e+10,30.804644,16.865612,31.509092,18.477237,11.883211
2959,2025-11-13,1631.44,1631.86,1638.98,1618.77,6.299500e+08,-0.03,46.111949,-18.227948,-13.485868,-4.742080,4.315525e+10,30.496416,17.589746,30.266649,32.751797,18.830393
2960,2025-11-14,1634.06,1625.55,1638.35,1623.03,7.039100e+08,0.16,46.650172,-16.781105,-14.144916,-2.636190,4.385916e+10,30.210206,17.041194,29.322756,43.642140,31.623725


In [25]:
# Optional: Save the dataframe with all indicators to CSV
output_path = "data/indicators/VNI_020114_141125_indicators.csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"Data with indicators saved to: {output_path}")


Data with indicators saved to: data/indicators/VNI_020114_141125_indicators.csv


In [22]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [26]:
# ============================================================================
# INTERACTIVE PLOT: VNI PRICE AND RSI_14
# ============================================================================
# This creates an interactive plot with two subplots:
# 1. Top panel: VNI Close Price over time
# 2. Bottom panel: RSI_14 indicator with overbought/oversold zones
#
# Features:
# - Zoom in/out: Use mouse wheel or box select
# - Pan: Click and drag
# - Reset: Double click
# - Hover: See exact values
# ============================================================================

# Create figure with 2 subplots (shared x-axis for time)
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=('VNI Index - Close Price', 'RSI (14-period)'),
    row_heights=[0.7, 0.3]
)

# ============================================================================
# SUBPLOT 1: VNI CLOSE PRICE
# ============================================================================
fig.add_trace(
    go.Scatter(
        x=df['Date'],
        y=df['Close'],
        name='VNI Close',
        line=dict(color='#2E86AB', width=2),
        hovertemplate='<b>Date</b>: %{x|%Y-%m-%d}<br>' +
                      '<b>Close</b>: %{y:,.2f}<br>' +
                      '<extra></extra>'
    ),
    row=1, col=1
)

# Add SMA lines for context
fig.add_trace(
    go.Scatter(
        x=df['Date'],
        y=df['SMA_20'],
        name='SMA 20',
        line=dict(color='#FFA500', width=1, dash='dash'),
        hovertemplate='<b>SMA 20</b>: %{y:,.2f}<extra></extra>'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df['Date'],
        y=df['SMA_50'],
        name='SMA 50',
        line=dict(color='#FF6B6B', width=1, dash='dash'),
        hovertemplate='<b>SMA 50</b>: %{y:,.2f}<extra></extra>'
    ),
    row=1, col=1
)

# ============================================================================
# SUBPLOT 2: RSI INDICATOR
# ============================================================================
fig.add_trace(
    go.Scatter(
        x=df['Date'],
        y=df['RSI_14'],
        name='RSI 14',
        line=dict(color='#8E44AD', width=2),
        hovertemplate='<b>Date</b>: %{x|%Y-%m-%d}<br>' +
                      '<b>RSI</b>: %{y:.2f}<br>' +
                      '<extra></extra>'
    ),
    row=2, col=1
)

# Add overbought line (70)
fig.add_hline(
    y=70, 
    line_dash="dash", 
    line_color="red", 
    opacity=0.7,
    annotation_text="Overbought (70)",
    annotation_position="right",
    row=2, col=1
)

# Add oversold line (30)
fig.add_hline(
    y=30, 
    line_dash="dash", 
    line_color="green", 
    opacity=0.7,
    annotation_text="Oversold (30)",
    annotation_position="right",
    row=2, col=1
)

# Add neutral line (50)
fig.add_hline(
    y=50, 
    line_dash="dot", 
    line_color="gray", 
    opacity=0.5,
    row=2, col=1
)

# Shade overbought zone (70-100)
fig.add_hrect(
    y0=70, y1=100,
    fillcolor="red", opacity=0.1,
    layer="below", line_width=0,
    row=2, col=1
)

# Shade oversold zone (0-30)
fig.add_hrect(
    y0=0, y1=30,
    fillcolor="green", opacity=0.1,
    layer="below", line_width=0,
    row=2, col=1
)

# ============================================================================
# LAYOUT CONFIGURATION
# ============================================================================
fig.update_layout(
    title={
        'text': '<b>VNI Index Technical Analysis - Price & RSI Indicator</b>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20}
    },
    height=800,
    hovermode='x unified',
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Update x-axes
fig.update_xaxes(
    title_text="Date",
    showgrid=True,
    gridwidth=1,
    gridcolor='lightgray',
    row=2, col=1
)

# Update y-axes
fig.update_yaxes(
    title_text="Price (VND)",
    showgrid=True,
    gridwidth=1,
    gridcolor='lightgray',
    row=1, col=1
)

fig.update_yaxes(
    title_text="RSI Value",
    showgrid=True,
    gridwidth=1,
    gridcolor='lightgray',
    range=[0, 100],
    row=2, col=1
)

# Display the interactive plot
fig.show()

print("\n✓ Interactive plot created successfully!")
print("\nHow to use:")
print("  • Zoom In: Click and drag to select an area")
print("  • Zoom Out: Double-click anywhere on the chart")
print("  • Pan: Click and drag while zoomed in")
print("  • Hover: Move mouse over lines to see values")
print("  • Toggle Lines: Click legend items to show/hide")



✓ Interactive plot created successfully!

How to use:
  • Zoom In: Click and drag to select an area
  • Zoom Out: Double-click anywhere on the chart
  • Pan: Click and drag while zoomed in
  • Hover: Move mouse over lines to see values
  • Toggle Lines: Click legend items to show/hide


In [27]:
df.columns

Index(['Date', 'Close', 'Open', 'High', 'Low', 'Volume', 'Change_Pct',
       'RSI_14', 'MACD', 'MACD_Signal', 'MACD_Hist', 'OBV', 'ADX', 'DI_Plus',
       'DI_Minus', 'STOCH_K', 'STOCH_D', 'SMA_20', 'SMA_50'],
      dtype='object')